# Hdfs remote connect 

In [1]:
#!pip install pyspark
# pyspark가 설치된 경로를 확인해서 SPARK_HOME 설정 

In [1]:
# !pip install findspark
# !pwd

In [2]:
# import libraries 
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
#import findspark
import time 

In [3]:
# 전역변수 설정 
# local mode 
SPARK_LOCAL_MASTER = "local[3]"
# client mode 
SPARK_CLUSTER_MASTER = "spark://34.125.237.158/:7077" 
SPARK_APP_NAME = "DATA-Preparation"
HOST_NAME = socket.gethostname()
PORT = 9999

# Define path
DATA_ROOT = f'{os.getcwd()}/data'
DATA_PATH = f'{DATA_ROOT}'

In [4]:
#os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
# os.environ['PATH'] = '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin/'
#os.environ['SPARK_HOME'] = '/usr/local'
#os.environ['SPARK_HOME']='/usr/local/lib/python3.6/dist-packages/pyspark'
# os.environ['HADOOP_HOME'] = '/hadoop-3.2.2'
# os.environ['hadoop.home.dir'] = '/hadoop-3.2.2/bin'
# os.environ['CLASSPATH'] = '$CLASSPATH:/hadoop-3.2.2/spark-3.2.0-bin-hadoop3.2.tar'
#findspark.init()
print(os.getenv('JAVA_HOME'))
print(os.getenv('SPARK_HOME'))

/usr/lib/jvm/java-8-openjdk-amd64/
/usr/local/lib/python3.6/dist-packages/pyspark


In [5]:
print(os.getenv('PATH'))
#findspark

/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin/


In [6]:
# Spark session 생성 메서드 
def init_remote_session():
    #SPARK_CLUSTER_MASTER = "spark://34.64.108.172:7077" 
    spark = SparkSession.builder.master(SPARK_CLUSTER_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    return spark
    
# local mode 실행 시 메모리를 확장해야 하는 경우 있어서, conf에 memory 변경 추가  
def init_local_session():
    #SPARK_LOCAL_MASTER = "local[3]"
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    default_conf = spark.sparkContext._conf#.getAll()
    print(f'Old Conf : {default_conf.getAll()}')
    conf = spark.sparkContext._conf.setAll([
        ('spark.executor.instances', 1)
        #, ('spark.driver.memory', '12g'), ('spark.executor.memory', '8g'), ('spark.driver.maxResultSize', '8g')
        , ('spark.driver.allowMultipleContexts', 'true'), ('spark.sql.shuffle.partitions', 8)
        ##,('spark.memory.offHeap.enabled', True), ('spark.memory.offHeap.size', '8g')
    ])
    spark.sparkContext.stop()
    
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config(conf=default_conf).getOrCreate()
    new_conf = spark.sparkContext._conf
    print(f'Updated Conf : {new_conf.getAll()}')
    return spark

In [7]:
!java -version

openjdk version "1.8.0_312"
OpenJDK Runtime Environment (build 1.8.0_312-8u312-b07-0ubuntu1~18.04-b07)
OpenJDK 64-Bit Server VM (build 25.312-b07, mixed mode)


In [8]:
# 스파크 생성 
spark = init_local_session()
# 스파크 클러스터를 실행해 두었다고 가정
#spark = init_remote_session()
sc = spark.sparkContext
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


Old Conf : [('spark.app.id', 'local-1644816980870'), ('spark.app.name', 'DATA-Preparation'), ('spark.sql.warehouse.dir', 'file:/notebooks/spark-warehouse'), ('spark.rdd.compress', 'True'), ('spark.driver.host', 'spark-client'), ('spark.master', 'local[3]'), ('spark.app.startTime', '1644816979731'), ('spark.serializer.objectStreamReset', '100'), ('spark.submit.pyFiles', ''), ('spark.driver.port', '34379'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]
Updated Conf : [('spark.sql.warehouse.dir', 'file:/notebooks/spark-warehouse'), ('spark.app.id', 'local-1644816982849'), ('spark.driver.host', 'spark-client'), ('spark.master', 'local[3]'), ('spark.app.startTime', '1644816982778'), ('spark.driver.allowMultipleContexts', 'true'), ('spark.executor.id', 'driver'), ('spark.app.name', 'DATA-Preparation'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.executor.instances', '1'), ('spark.su

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=local[3] appName=DATA-Preparation>

# HDFS Run
Spark Driver와 다른 위치에서 Hadoop cluster를 실행하고, 이를 연결해서 사용해 본다.  
172.17.0.3:9000 에 Hadoop 머신을 실행해 둔다.  

In [ ]:
spark.read.textFile("hdfs://172.17.0.3:9000/user/root/output").textDF.show()